In [12]:
# Our numerical workhorses
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tifffile as tif
import seaborn as sns
from scipy.ndimage.filters import gaussian_laplace, gaussian_filter1d, gaussian_filter

# This is to enable inline displays for the purposes of the tutorial
%matplotlib inline

# This enables SVG graphics inline
%config InlineBackend.figure_formats = {'png', 'retina'}

rc = {'lines.linewidth': 2, 'axes.labelsize': 18,  'axes.titlesize': 18, 'axes.facecolor': 'DFDFE5'}
sns.set_context('notebook', rc=rc)
sns.set_style('white')

# Suppress future warnings
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [60]:
def image_coloredpoints2(image,x,y,color,z):
    # image: np.zeros([2048,2048,3],dtype=np.uint8)
    # x,y: rounded values
    x = int(x)
    y = int(y)
    z = int(z)
    image[z,x-1,y-1] = color
    image[z,x-1,y] = color
    image[z,x-1,y+1] = color
    image[z,x,y-1] = color
    image[z,x,y] = color
    image[z,x,y+1] = color
    image[z,x+1,y-1] = color
    image[z,x+1,y] = color
    image[z,x+1,y+1] = color
    
    return image

In [85]:
# selected loci
loci_names = ['Ttn','chr2-#15','Fmnl2','chr4-#67','Calb1','Id3','Nanog','Grid2','Gapdh','Dnmt3l','Tcf3','Grik2','Pou5f1',
'Ehmt2','Nox3',             
'Nrxn2',
'Cnih2',
'Zfand5',
'Aldoc',
'Foxj1',
'Efemp1']

print(len(loci_names))

# specific FOV ID
fov_id = 3

21


In [86]:
# main DNA seqFISH+ file found in Zenodo
file_path = 'E:/Yodai/DNAFISH+/Zenodo/DNAseqFISH+/'


df = pd.read_csv(file_path+'DNAseqFISH+1Mbloci-E14-replicate2.csv')
df = df[(df['fov']==fov_id)&((df['seeds']==4)|(df['seeds']==5))&(df['geneID'].isin(loci_names))].reset_index(drop=True)

print(len(df))

df.head()

1731


,Unnamed: 0,fov,cellID,channel,geneID,x,y,z,seeds,intensity
0,259686,3,1,1,chr2-#15,1353.393,621.990,8.633,4,1353
1,259687,3,1,1,chr2-#15,1249.681,621.867,7.047,4,1250
2,260097,3,1,1,chr4-#67,1312.103,641.879,10.122,5,1312
3,261829,3,2,1,chr2-#15,1297.122,705.605,9.619,5,1297
4,261830,3,2,1,chr2-#15,1310.141,790.204,8.694,4,1310


### creat RGB images with pseudo-dots of selected loci based on DNA seqFISH+ voxel location downloaded from Zenodo

each tif file per each locus

manually convert to 16-bit images by ImageJ (switch to 8 bit then 16 bit)

then merge with 16-bit IF image below

In [87]:
#fig,ax = plt.subplots(1,1,figsize=(10,10))

# plot whole z sections
z_min = 1
z_max = 25

for j in range(18,len(loci_names)):
    
    df_fp = df[df['geneID']==loci_names[j]].reset_index(drop=True)

    # make 8 bit blank images
    image_fp1 = np.zeros([25,2048,2048,3],dtype=np.uint8) # blank image with actual image size

    # raw dots on the entire blank image based on the rounded DNA seqFISH+ spot location
    for i in range(len(df_fp)):
        if np.round(df_fp['z'][i]) >= z_min and np.round(df_fp['z'][i]) <= z_max:
            image_fp1 = image_coloredpoints2(image_fp1,np.round(df_fp['y'][i])-1,
                                           np.round(df_fp['x'][i])-1,[255, 255, 255],np.round(df_fp['z'][i])-1)

    # gaussian blur of the dots with defined sigma
    image_gf1 = gaussian_filter(image_fp1,sigma=1)

    # just for visualization
    #ax.imshow(image_gf1)

    tif.imwrite('output_Prashant/locus_'+loci_names[j]+'_fov'+str(fov_id)+'-replicate2.tif',image_gf1, compression='DEFLATE')

### create SF3A66 immunofluorescence image from single-cell csv files downloaded from Zenodo

In [58]:
if_path = 'F:/Yodai/DNA+/2019-08-03-E14-DNA-seqFISH+rep3-1-plate2/IF-binned1x1x1-E14-replicate2/IF-binned1x1x1-E14-replicate2-Pos'+str(fov_id)+'/'

In [59]:
a = np.zeros([1,25,2048,2048],dtype=np.uint16) # blank image with actual image size

for c in range(51): # number of cell per FOV (manually added for now)
    df_if = pd.read_csv(if_path+'outputData-ImmunoFluorescence-Ch2-Pos'+str(fov_id)+'-Cell'+str(c+1)+'-2020-02-07.csv')
    df_if = df_if[df_if['hybID']==8].reset_index(drop=True) # SF3A66 hyb number (rep specific, based on the file in Zenodo)

    for i in range(len(df_if)):
        a[0,int(df_if['zIdx'][i])-1,int(df_if['yIdx'][i])-1,int(df_if['xIdx'][i])-1] = int(df_if['avgint'][i])
    
tif.imwrite('output_Prashant/SF3A66_pos'+str(fov_id)+'-replicate2.tif',a, compression='DEFLATE')